In [1]:
# # Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
from Functions import *

In [3]:
datamodel = load_data()
datamodel.head()


,ÉLIGIBILITÉ AU DON.,Age,Taux _hémoglobine_(g/dl),Genre_Homme,Good_profession_Chomeurs,Good_profession_Employes de type administratif,Good_profession_Intellectuels et scientifiques,Good_profession_Non précisée,"Good_profession_Personnel des services directs aux particuliers, commercants vendeurs",Good_profession_artisants et ouvriers d'industrie,"Good_profession_dirigeants,cadre de direction et gerants",Good_profession_forces de defense et securité personnel,Good_profession_professions intermediaires,Good_profession_Élève,ancien_don_sang_Oui
0,1,-0.003865,-0.086059,1,0,0,0,0,0,0,0,0,1,0,0
1,1,-0.003865,-0.086059,1,0,0,0,0,0,0,0,0,1,0,0
2,1,-0.906069,1.290946,1,0,0,0,0,0,0,0,0,1,0,1
3,1,-0.390524,-1.245641,0,0,0,0,0,0,0,0,0,1,0,0
4,1,-0.648296,1.725790,1,0,0,1,0,0,0,0,0,0,0,0


In [4]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(datamodel)

In [5]:
best_model, best_params = optimize_rforest(X_train, y_train, X_val, y_val)

Meilleurs paramètres: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 90}
Précision sur l'ensemble de validation: 0.8972


In [6]:
evaluate_model(best_model, X_train, X_val, X_test, y_train, y_val, y_test)

(0.9003516998827668,
 0.897196261682243,
 0.8878504672897196,
               precision    recall  f1-score     support
 0              0.944444  0.515152  0.666667  165.000000
 1              0.895151  0.992733  0.941420  688.000000
 accuracy       0.900352  0.900352  0.900352    0.900352
 macro avg      0.919798  0.753942  0.804043  853.000000
 weighted avg   0.904686  0.900352  0.888273  853.000000,
               precision    recall  f1-score     support
 0              1.000000  0.576923  0.731707   26.000000
 1              0.880435  1.000000  0.936416   81.000000
 accuracy       0.897196  0.897196  0.897196    0.897196
 macro avg      0.940217  0.788462  0.834062  107.000000
 weighted avg   0.909488  0.897196  0.886674  107.000000,
               precision    recall  f1-score    support
 0              1.000000  0.478261  0.647059   23.00000
 1              0.875000  1.000000  0.933333   84.00000
 accuracy       0.887850  0.887850  0.887850    0.88785
 macro avg      0.937500  0.

In [12]:
# Exemple de prédiction pour un nouvel individu
print("\nExemple de prédiction pour un nouvel individu:")
new_individual = {
    'Age': 18,
    'Taux_hemoglobine': 12,
    'Genre': 2,  # Homme
    'Good_profession': 2,  # Catégorie professionnelle 2
    'ancien_don_sang': 1  # A déjà donné du sang
}

prediction_result = predict_new_individual(new_individual,choice=2)
print(f"Prédiction: {prediction_result}")


Exemple de prédiction pour un nouvel individu:
Prédiction: {'prediction': 1, 'probability': 0.6535050328480788, 'interpretation': 'Éligible'}


In [ ]:
def optimize_rforest(X_train, y_train, X_val, y_val):
      
    # Meilleurs paramètres 
    best_params = {'bootstrap': False,
                'criterion': 'entropy',
                'max_depth': 5,
                'max_features': 'sqrt',
                'min_samples_leaf': 1,
                'min_samples_split': 5,
                'n_estimators': 90}
    
    print(f"Meilleurs paramètres: {best_params}")
   
    # Initialisation du modèle avec les meilleurs paramètres
    best_model = RandomForestClassifier(
            bootstrap = False,
            criterion = 'entropy',
            max_depth = 5,
            max_features = 'sqrt',
            min_samples_leaf = 1,
            min_samples_split = 5,
            n_estimators = 90)
    best_model.fit(X_train, y_train)
    
    # Évaluation sur l'ensemble de validation
    val_accuracy = best_model.score(X_val, y_val)
    print(f"Précision sur l'ensemble de validation: {val_accuracy:.4f}")
    # Sauvegarde du modèle
    joblib.dump(best_model, 'random_forest_model.pkl')
    
    return best_model, best_params